In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dir_path = "/content/drive/My Drive/download/"

In [0]:
from glob import glob
from os import getcwd, chdir
 
def list_files(directory, extension):
    saved = getcwd()
    chdir(directory)
    it = glob('*.' + extension)
    chdir(saved)
    return it

In [0]:
X = list_files(dir_path,"wav")
X

['03a01Nc.wav',
 '03a01Wa.wav',
 '03a02Wb.wav',
 '03a02Wc.wav',
 '03a02Nc.wav',
 '03a04Nc.wav',
 '03a04Ta.wav',
 '03a01Fa.wav',
 '03a02Ta.wav',
 '03a04Lc.wav',
 '03a04Ad.wav',
 '03a04Fd.wav',
 '03a02Fc.wav']

In [0]:
# W	anger
# L	boredom
# E	disgust
# A	anxiety/fear
# F	happiness
# T	sadness
# N = neutral version


Y = []
for f in X:
  if(f[5]=='N'):
    Y.append(0)
  
  if(f[5]=='W'):
    Y.append(1)

  if(f[5]=='L'):
    Y.append(2)

  if(f[5]=='E'):
    Y.append(3)
  
  if(f[5]=='A'):
    Y.append(4)

  if(f[5]=='F'):
    Y.append(5)
  
  if(f[5]=='T'):
    Y.append(6)

# Y



[0, 1, 1, 1, 0, 0, 6, 5, 6, 2, 4, 5, 5]

In [0]:
import numpy as np
import scipy.io.wavfile
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.fftpack import dct
from scipy import stats

In [0]:
def normlize(x):
    return ((x-np.min(x))/(np.max(x)-np.min(x)))

In [0]:
def getFeatures(filename):
  filepath = dir_path + filename

  sample_rate, signal = scipy.io.wavfile.read(filepath)  
  
  signal = signal[0:int(3.5 * sample_rate)]  # Keep the first 3.5 seconds

  pre_emphasis = 0.95
  emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])


  frame_size = 0.025
  frame_stride = 0.01

  frame_length = frame_size * sample_rate # Convert from seconds to samples
  frame_step = frame_stride * sample_rate

  signal_length = len(emphasized_signal)
  frame_length = int(round(frame_length))
  frame_step = int(round(frame_step))
  num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

  pad_signal_length = num_frames * frame_step + frame_length
  z = np.zeros((pad_signal_length - signal_length))
  pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

  indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
  frames = pad_signal[indices.astype(np.int32, copy=False)]

  frames *= np.hamming(frame_length)

  NFFT = 512
  mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
  pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

  nfilt = 40

  low_freq_mel = 0
  high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
  mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
  hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
  bin = np.floor((NFFT + 1) * hz_points / sample_rate)

  fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
  for m in range(1, nfilt + 1):
      f_m_minus = int(bin[m - 1])   # left
      f_m = int(bin[m])             # center
      f_m_plus = int(bin[m + 1])    # right

      for k in range(f_m_minus, f_m):
          fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
      for k in range(f_m, f_m_plus):
          fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
  filter_banks = np.dot(pow_frames, fbank.T)
  filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
  filter_banks = 20 * np.log10(filter_banks)  # dB

  static = filter_banks
  delta = librosa.feature.delta(static, order=1)
  delta2 = librosa.feature.delta(static, order=2)

  static = normlize(static)
  delta = normlize(delta)
  delta2 = normlize(delta2)

  # print(static)

  # static = np.array(static)
  # delta = np.array(delta)
  # delta2 = np.array(delta2)

  # librosa.display.specshow(static)
  # librosa.display.specshow(delta)
  # librosa.display.specshow(delta2)

  cnn_input = np.dstack((static,delta,delta2))

  # cnn_input = np.array(cnn_input)
  

  return cnn_input



In [0]:
x = []
y = Y
y = np.array(y)

for f in X:
  temp  = getFeatures(f)
  temp = np.resize(temp,(160,40,3))
  x.append(temp)

x = np.array(x)

print(x.shape)
print(y.shape)





(13, 160, 40, 3)
(13,)


In [0]:
from sklearn.model_selection import KFold
k = 10

In [0]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models 
from tensorflow.keras.layers import Dense , Reshape ,Activation


In [0]:
for train_index,test_index in KFold(k).split(x):

  x_train,x_test=x[train_index],x[test_index]
  y_train,y_test=y[train_index],y[test_index]

  # print(x_train.shape)
  # print(x_test.shape)
  # print(y_train.shape)
  # print(y_test.shape)

  model = models.Sequential()
  model.add(layers.Conv2D(128, (5, 3), activation='relu', input_shape=(160, 40, 3)))
    

  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))


  model.add(layers.Conv2D(256, (5, 3),activation='relu'))
    
  model.add(layers.Dropout(0.2))
    
  model.add(layers.Conv2D(256, (5, 3),activation='relu'))
    
  model.add(layers.Dropout(0.2))

  model.add(layers.Conv2D(256, (5, 3),activation='relu'))

  model.add(layers.Dropout(0.2))

  model.add(layers.Conv2D(256, (5, 3),activation='relu'))

  model.add(layers.Dropout(0.2))

  model.add(layers.Flatten())
  model.add(Dense(768, activation='relu', kernel_initializer='he_uniform'))


  model.add(Reshape((1,768)))

  model.add(layers.Bidirectional(layers.LSTM(128)))


  model.add(Dense(64))
  model.add(layers.BatchNormalization())
  model.add(Dense(7))
  
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  model.fit(x_train, y_train,epochs=10)
  
  print('Model evaluation ',model.evaluate(x_test,y_test))

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 1.9473 - accuracy: 0.0909
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: 2.1673 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 1ms/step - loss: 2.1550 - accuracy: 0.1818
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 1.9120 - accuracy: 0.1818
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 1.6034 - accuracy: 0.3636
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 1.9680 - accuracy: 0.2727
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 1.7017 - accuracy: 0.3636
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 2.2324 - accuracy: 0.0909
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 1.8112 - accuracy: 0.2727
Epoch 10/10
1/1 [==============================] - 0s 2ms/step - loss: 2.5049 - accuracy: 0.0000e+00
Model evaluation  [2.5049285888

In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [0]:
predictions = probability_model.predict(x_test)

In [0]:
predictions[0]

array([0.08073713, 0.05234797, 0.32240772, 0.09097443, 0.34852213,
       0.05844668, 0.04656399], dtype=float32)

In [0]:
np.argmax(predictions[0])

4